## 3D Feature Matching Progress Notes

### Objective
The objective of 3D feature matching is to match 3D features which are known as a multi-view track in SfM pipeline from different point clouds. Based on the point correspondences, merging multiple SfM results is made possible by estimating reliable (similarity) transformations between SfM blocks. As the first yet non-trivial step, learning expressive and discriminative descriptors of 3D tracks is of utmost importance.

### Learn Descriptors
Different from pairwise matching, matching 3D tracks is concerned about matching about two groups of views. The representation of each track therefore should integrate the describing capability of various views. The local patches of different views usually undergoes some extent of variance (at least affine transformation). From this perspective, the essential goal of learning is associated with learning the variance of multi-view patches. Generally speaking, the higher nonlinearity the model present, the more abtract activations are produced, and the larger invariance is achieved. For example, in the context of image classification, the very abstract class information like Dog tolerates wide spread of variance of species and appreance of dogs. It's the invariant property that determines the accuracy of classification.

#### Progress
1. [19/04/2017~20/04/2017] The intial try just takes the siamese network of matchnet and utilize the feature maps on the top of feature tower as the descriptor. The dimension is 4096 for now. Although the weights are trained with a metric network hanging above, the recall of matching with the L2 distance of top feature map is higher than that of matching by the metric network (**52/81(L2) VS 31/81(metric) for stadium**). This makes sense that the overloaded fully connected network is not suitable for patch matching. As inspired by SIFT, a good descriptor can be achieved by aggregating the charateristics of local sub-regions (histogram of gradients in 8 directions are aggregated from 4x4 sub-regions). It is noteworthy that the L2 distances of the 4096-dimensional descriptors are relative. Although filtering the mismatches by absolute distance is impossible here, ratio test is appliable to rule out mismatches. As observed from experimants, outlier matches generally own distance ratio (ratio of distances of closet and second closet neighbors) near 1, while inlier matches can have distance ratio far away from 1 as shown below. For our task of 3D matching, precision of 3D feature correspodences is much more important than recall. Hence, filtering putative matches by ratio test does more good than harm although some inlier matches are also discarded.
![Distribution of Distance Ratios](images/dist_ratio.png)
![SFZ](images/SFZ.png)
![gammon](images/gammon.png)

2. **Normalization of feature map** Generally, $C$ channels of output feature maps denote the activations of certain distinctive pattern in input image. Assume $C$ feature functions are composed of cascading convolutions, rectified linear units and poolings, then the top feature map in each channel produced by the feature functions characterizes the feautures that are beyond description due to the high nonlinearity. However, in practice, the activations of some channels are small while activations of some are large. In this case, the 'heavier' channels would dominate the distance measurement and the activations from the 'lighter' channels are wasted. To mitigate the effect, I resort to rescaling methods (MinMax or L2 normalization) to **rectify or inhibit** activations within channels. Based on this, L2 distances are computed and improvements are observed (**1523/4000(MinMax) VS 1491/4000(L2) VS 1416/4000(None) for stadium**). What's more, the positive and negative matches become more separable by their distance:
![Distribution of disrances](images/distances.png)

3. [20/04/2017~] The objective of this stage is to train the absolute and uniform distance measurement for matching so that correspondences with too large distances can be filtered out uniformly. In order to train absolute distance and preserve contrast between positive and negative pairs simutaneously, constrastive loss is adopted. 
    * **Single margin VS Bi-margin** Although it is desired that similar pairs exhibit zero distance in space, it is intractable in practice. In order to make life easier, an extra margin_simi is added to distances of postive pairs so that the distance of positive pairs is computed by max(0, margin_simi-dist).
A bottleneck is put onto the dim-4096 feature map to reduce the dimension to 512, followed by an L2 normalization layer. For the stadium dataset, **single-margin (0, 0.3)** gives recall **306/4000** which indicates very poor performance. On the contrary, **bi-margin setting (0.2, 0.3)** gives recall **1357/4000**, which is a little bit lower than 2, which is acceptable considering lower dimension. However, there are still two problems. (1) One is that the learned descriptors are still not discriminative enough. Even a lot of mismatches exhibit very small L2 distances near zero. Therefore, mining hard negatives is of utmost importance. (2) The second thing that needs further exploration is the setting of two margins.

        * **Margin Setting** To evaluate the influence of margin setting on feature discriminativeness, another setting **(0.1, 0.4)** is adopted and trained based on the model trained from (0.2, 0.3). Numerical comparison of descriptor matching of the earthquake dataset is shown in table below. (0.1, 0.4) outperforms (0.2, 0.3). It implies that separating the margins more widely indeed helps in increasing the discriminative power of descriptors. A simple way of defining the discriminative power of descriptors in matching tasks is that
        
        if match is true, $$discriminativeness = log \frac{D_{sec\_min}}{D_{min}};$$
        else if match is false, $$discriminativeness = 0.$$


|    | Siamese+metric 4096 pooling feature map | Siamese+contrastive 512 fc feature map Margin: 0.2, 0.3 | Siamese+contrastive 512 fc feature map Margin: 0.1, 0.4 |Ground truth| 
|:---------:|:-------------:|:---------------------:|:-------------------:|:-----------:|:-----------:|
| #TP matches (Earthquake) | 32       | 23      | **33**      |39|
| #TP matches (SFZ) | 469       | 435      | **708**      |990|
| #TP matches (shaoguan) | 2093       | 1563      | ~~2247~~  **2755**    |5128| 
|Avg discriminative power (shaoguan)| 0.491 | 0.601 | ~~0.738~~ **0.861**|




